### Necessary imports and utility functions

In [47]:
import numpy as np
from Bio.Align import substitution_matrices

# Type hinting
from typing import List, Set, Union, Callable
# type sim_fct = Callable[[chr, chr], float]

blossum = substitution_matrices.load("BLOSUM62")
print(blossum)


def sim_blossum(a:chr, b:chr, gap=-8)->float:
    if a == '-' or b == '-':
        return gap
    return blossum[a,b]

def sim_basic(a:chr, b:chr, id=1, sub=-1, gap=-2)->float:
    if a == b:
        return id
    elif a == '-' or b == '-':
        return gap
    else:
        return sub

#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
     A    R    N    D    C    Q    E    G    H    I    L    K    M    F    P    S    T    W    Y    V    B    Z    X    *
A  4.0 -1.0 -2.0 -2.0  0.0 -1.0 -1.0  0.0 -2.0 -1.0 -1.0 -1.0 -1.0 -2.0 -1.0  1.0  0.0 -3.0 -2.0  0.0 -2.0 -1.0  0.0 -4.0
R -1.0  5.0  0.0 -2.0 -3.0  1.0  0.0 -2.0  0.0 -3.0 -2.0  2.0 -1.0 -3.0 -2.0 -1.0 -1.0 -3.0 -2.0 -3.0 -1.0  0.0 -1.0 -4.0
N -2.0  0.0  6.0  1.0 -3.0  0.0  0.0  0.0  1.0 -3.0 -3.0  0.0 -2.0 -3.0 -2.0  1.0  0.0 -4.0 -2.0 -3.0  3.0  0.0 -1.0 -4.0
D -2.0 -2.0  1.0  6.0 -3.0  0.0  2.0 -1.0 -1.0 -3.0 -4.0 -1.0 -3.0 -3.0 -1.0  0.0 -1.0 -4.0 -3.0 -3.0  4.0  1.0 -1.0 -4.0
C  0.0 -3.0 -3.0 -3.0  9.0 -3.0 -4.0 -3.0 -3.0 -1.0 -1.0 -3.0 -1.0 -2.0 -3.0 -1.0 -1.0 -2.0 -2.0 -1.0 -3.0 -3.0 -2.0 -4.0
Q -1.0  1.0  0.0  0.

# Aligning two sequences

### Needleman-Wunsch: alignment with linear gap cost
This algorithm tries to align to protein sequences $a_1^n$ and $b_1^m$.
By dynamic programming, the algorithm computes the maximum similarity between any pair of prefixes $a_{1...i}$ and $b_{1...j}$ and stores them in a matrix $M_{i,j}$

$M_{i,j}$ is defined by recursion with the following formula:
$$
M_{i,j} = \max \begin{cases}
  M_{i-1,j-1} + s(a_i,b_j) \\
  M_{i-1,j} - \text{gap cost} &\\
  M_{i,j-1} - \text{gap cost}&
\end{cases}
$$

To facilitate the traceback, we also compute a matrix $D$ *(like Directions)* indicating which case achieved the maximum.

In [48]:
def compute_matrix_needleman_wunsch(a:str, b:str, sim_fct):
    """ compute the Needleman-Wunsch matrix """

    # construct matrices
    M = np.zeros((len(a)+1, len(b)+1), dtype=int)
    D = np.zeros((len(a)+1, len(b)+1), dtype=int)

    # init matrices
    for i in range(1, len(a)+1):
        M[i,0] = M[i-1, 0] + sim_fct(a[i-1], '-')
    for j in range(1, len(b)+1):
        M[0,j] = M[0, j-1] + sim_fct('-', b[j-1])

    # fill matrices
    for i in range(1, len(a)+1):
        for j in range(1, len(b)+1):
            x1 = M[i-1,j-1]+sim_fct(a[i-1], b[j-1])
            x2 = M[i-1,j]+sim_fct(a[i-1], '-')
            x3 = M[i,j-1]+sim_fct('-', b[j-1])
            if x1 >= x2 and x1 >= x3: # diagonal
                M[i,j] = x1
                D[i,j] = 1
            if x2 >= x1 and x2 >= x3: # up
                M[i,j] = x2
                D[i,j] = 2
            if x3 >= x1 and x3 >= x2: # left
                M[i,j] = x3
                D[i,j] = 3
    
    return M, D

def trace_back(a:str, b:str, D:np.array):
    """ traceback matrix A to get the alignment """
    aa = ""
    bb = ""
    
    i = len(a)
    j = len(b)
    while i>0 and j>0 :
        if D[i,j] == 1:
            aa = a[i-1] + aa
            bb = b[j-1] + bb
            i -= 1
            j -= 1
        elif D[i,j] == 2:
            aa = a[i-1] + aa
            bb = '-' + bb
            i -= 1
        else:
            aa = '-' + aa
            bb = b[j-1] + bb
            j -= 1
    if i == 0:
        aa = '-'*j + aa
        bb = b[:j] + bb
    elif j == 0:
        aa = a[:i] + aa
        bb = '-'*i + bb

    return aa, bb

def needleman_wunsch_global_alignment(a:str, b:str, sim_fct=sim_blossum):
    """ Global alignment with linear gap cost """
    M, A = compute_matrix_needleman_wunsch(a, b, sim_fct)
    aa, bb = trace_back(a, b, A)
    print(aa)
    print(bb)
    return M, aa, bb

In [49]:
# test
A = "CHATS"
B = "CAT"

F, aa, bb = needleman_wunsch_global_alignment(A, B, sim_blossum)
print(F)

CHATS
C-AT-
[[  0  -8 -16 -24]
 [ -8   9   1  -7]
 [-16   1   7  -1]
 [-24  -7   5   7]
 [-32 -15  -3  10]
 [-40 -23 -11   2]]


### Gotoh: alignment with affine gap cost
Instead of one matrix, we need 3:
- $F_{i,j}$ storing the best score for $a_{1...i}$ and $b_{1...j}$ that ends with $a_i$ and $b_j$
- $A_{i,j}$ storing the best score that ends with $a_i$ and a gap
- $B_{i,j}$ storing the best score that ends with a gap and $b_j$

We have:
$$
A_{i,j} = \max \begin{cases}
  A_{i-1,j} + \text{gap extension} \\
  F_{i-1,j} + \text{gap opening} &\\
\end{cases}
$$

$$
B_{i,j} = \max \begin{cases}
  B_{i,j-1} + \text{gap extension} \\
  F_{i,j-1} + \text{gap opening} &\\
\end{cases}
$$
$$
F_{i,j} = \max \begin{cases}
  F_{i-1,j-1} + s(a_i,b_j)  \\
  F_{i-1,j-1} + s(a_i,b_j)  \\
  F_{i-1,j-1} + s(a_i,b_j)  \\
\end{cases}
$$

The matrix $M_{i,j}$ as defined previously is the max (element wise) of this three matrices.

In [50]:
def compute_matrix_gotoh(a:str, b:str, sim_fct, gap_opening:float, gap_extension:float):
    """ compute the Gotoh matrices """
    F = np.zeros((len(a)+1, len(b)+1))
    A = np.zeros((len(a)+1, len(b)+1))
    B = np.zeros((len(a)+1, len(b)+1))

    # init matrix A
    for j in range(0, len(b)+1):
        A[0,j] = -np.inf
        A[1,j] = 2*gap_opening + (j-1)*gap_extension
    for i in range(1, len(a)+1):    
        A[i,0] = gap_opening + (i-1)*gap_extension

    # init matrix B
    for i in range(0, len(a)+1):
        B[i,0] = -np.inf
        B[i,1] = 2*gap_opening + (i-1)*gap_extension
    for j in range(1, len(b)+1):
        B[0,j] = gap_opening + (j-1)*gap_extension

    # init matrix F
    F[0,0] = 0
    for i in range(1, len(a)+1):
        F[i,0] = -np.inf
    for j in range(1, len(b)+1):
        F[0,j] = -np.inf
        F[0,j] = -np.inf    

    # fill matrices
    for i in range(1, len(a)+1):
        for j in range(1, len(b)+1):
            if i>1:
                A[i,j] = max(F[i-1,j]+gap_opening, A[i-1,j]+gap_extension)
            if j>1:
                B[i,j] = max(F[i,j-1]+gap_opening, B[i,j-1]+gap_extension)
            F[i,j] = max(F[i-1,j-1]+sim_fct(a[i-1], b[j-1]), A[i-1,j-1]+sim_fct(a[i-1], b[j-1]), B[i-1,j-1]+sim_fct(a[i-1], b[j-1]))
    
    return F, A, B

def trace_back_gotoh(a:str, b:str, F:np.array, A:np.array, B:np.array):
    """ Does the traceback using the matrices M, A and B
    To know in wich direction to go, we look which matrix has the highest value"""
    aa = ""
    bb = ""
    i = len(a)
    j = len(b)
    while i>0 and j>0 :
        m = max(F[i,j], A[i,j], B[i,j])
        if F[i,j] == m:     # diagonal
            aa = a[i-1] + aa
            bb = b[j-1] + bb
            i -= 1
            j -= 1
        elif A[i,j] == m:   # up
            aa = a[i-1] + aa
            bb = '-' + bb
            i -= 1
        else:               # left
            aa = '-' + aa
            bb = b[j-1] + bb
            j -= 1
    if i == 0:
        aa = '-'*j + aa
        bb = b[:j] + bb
    elif j == 0:
        aa = a[:i] + aa
        bb = '-'*i + bb

    return aa, bb

def gotoh_global_alignment(a:str, b:str, sim_fct=sim_basic, gap_opening=-2, gap_extension=-2):
    """ Global alignment with affine gap cost """
    F, A, B = compute_matrix_gotoh(a, b, sim_fct, gap_opening, gap_extension)
    aa, bb = trace_back_gotoh(a, b, F, A, B)
    print(aa)
    print(bb)
    return np.maximum(F, np.maximum(A,B)), aa, bb

In [51]:
# test
A = "CHAT"
B = "CAT"

M, aa, bb = gotoh_global_alignment(A, B, sim_basic, -2, -2)
print(M)

CHAT
C-AT
[[ 0. -2. -4. -6.]
 [-2.  1. -1. -3.]
 [-4. -1.  0. -2.]
 [-6. -3.  0. -1.]
 [-8. -5. -2.  1.]]


# Aligning N sequences

### Pairwise profile alignment
We want to merge two alignments $A$ and $B$ into one by aligning them. For this, we can modify the Needleman-Wunsch algorithm. We just have to adapt the cost function of an alignment, to correspond to the average cost over each pair $(a,b) \in (A,B)$.


In [52]:
def sim_multiple(A:List[str], B:List[str], i:int, j:int, sim_fct):
    """ Compute the average similarity between two sets of aligned sequences, at a given position """
    res = 0
    for a in A:
        for b in B:
            res += sim_fct(a[i],b[j])
    return res/(len(A)*len(B))

def compute_matrix_needleman_wunsch_multiple(A:List[str], B:List[str], sim_fct):
    """ Compute the Needleman-Wunsch dynamic programming matrix for two sets of aligned sequences """
    len_a = len(A[0])
    len_b = len(B[0])

    # construct matrices
    M = np.zeros((len_a+1, len_b+1))
    D = np.zeros((len_a+1, len_b+1), dtype=int)

    # init matrices
    for i in range(1, len_a+1):
        M[i,0] = i * sim_fct('a', '-')  # cost of a gap
    for j in range(1, len_b+1):
        M[0,j] = j * sim_fct('a', '-')

    # fill matrices
    for i in range(1, len_a+1):
        for j in range(1, len_b+1):
            x1 = M[i-1,j-1] + sim_multiple(A, B, i-1, j-1, sim_fct)
            x2 = M[i-1,j] + sim_fct('a', '-')
            x3 = M[i,j-1] + sim_fct('a', '-')
            if x1 >= x2 and x1 >= x3: # diagonal
                M[i,j] = x1
                D[i,j] = 1
            elif x2 >= x1 and x2 >= x3: # up
                M[i,j] = x2
                D[i,j] = 2
            else: # x3 >= x1 and x3 >= x2: # left
                M[i,j] = x3
                D[i,j] = 3
    
    return M, D


def trace_back_multiple(A:List[str], B:List[str], D:np.array):
    """ Determine the alignment given a matrix with directions """

    AA = ["" for _ in range(len(A))]
    BB = ["" for _ in range(len(B))]
    
    i = len(A[0])
    j = len(B[0])
    while i>0 and j>0 :
        if D[i,j] == 1:
            for a in range(len(A)):
                AA[a] = A[a][i-1] + AA[a]
            for b in range(len(B)):
                BB[b] = B[b][j-1] + BB[b]
            i -= 1
            j -= 1
        elif D[i,j] == 2:
            for a in range(len(A)):
                AA[a] = A[a][i-1] + AA[a]
            for b in range(len(B)):
                BB[b] = '-' + BB[b]
            i -= 1
        else:
            for a in range(len(A)):
                AA[a] = '-' + AA[a]
            for b in range(len(B)):
                BB[b] = B[b][j-1] + BB[b]
            j -= 1
    if i == 0:
        for a in range(len(A)):
            AA[a] = '-'*j + AA[a]
        for b in range(len(B)):
            BB[b] = B[b][:j] + BB[b]
    elif j == 0:
        for a in range(len(A)):
            AA[a] = A[a][:i] + AA[a]
        for b in range(len(B)):
            BB[b] = '-'*i + BB[b]

    return AA + BB   # list concatenation

In [53]:
# test
A = "CHAT"
B = "CAT"

M, D = compute_matrix_needleman_wunsch(A, B, sim_blossum)
print(M)
print()

M, D = compute_matrix_needleman_wunsch_multiple([A], [B], sim_blossum)
print(M)

L = trace_back_multiple([A], [B], D)
print(L)

[[  0  -8 -16 -24]
 [ -8   9   1  -7]
 [-16   1   7  -1]
 [-24  -7   5   7]
 [-32 -15  -3  10]]

[[  0.  -8. -16. -24.]
 [ -8.   9.   1.  -7.]
 [-16.   1.   7.  -1.]
 [-24.  -7.   5.   7.]
 [-32. -15.  -3.  10.]]
['CHAT', 'C-AT']


###  Progressive multiple alignment
To align N sequences, we do clustering. First, we create one cluster per sequence. Then, we align and merge the two closest clusters until we only have one big cluster, corresponding to the alignment of all sequences.

In [54]:
from time import time

def progressive_alignment(seq:List[str], sim_fct): # TODO: check if possible to improve efficiency
  """ Computes multiple sequence alignement """
  time_mat = 0
  time_trace = 0

  # transforming each sequence into a cluster of one sequence
  clusters = [[s] for s in seq]  

  # initialising distance matrix
  matrix = np.zeros((len(seq), len(seq)))
  for i in range(len(seq)):
    for j in range(i+1, len(seq)):
      start = time()
      matrix[i,j] = compute_matrix_needleman_wunsch(seq[i], seq[j], sim_fct)[0][-1,-1]
      time_mat += time() - start

  while len(clusters) > 1:
    # finding closest clusters
    max_sim = -np.inf
    for i in range(len(clusters)):
      for j in range(i+1,len(clusters)):
        if matrix[i,j] > max_sim:
          max_sim = matrix[i,j]
          best_i = i
          best_j = j

    # merging two clusters best_i < best_j
    a = clusters[best_i]
    b = clusters[best_j]
    start = time()
    D, A = compute_matrix_needleman_wunsch_multiple(a, b, sim_fct)
    time_mat += time() - start
    start = time()
    new_cluster = trace_back_multiple(a, b, A)
    time_trace += time() - start

    clusters.remove(a)
    clusters.remove(b)
    clusters.append(new_cluster)

    # updating distance matrix
    matrix = np.delete(matrix, [best_i,best_j], axis=0)
    matrix = np.delete(matrix, [best_i,best_j], axis=1)

    new_column = []
    for cluster in clusters[:-1]:
      start = time()
      new_column.append(compute_matrix_needleman_wunsch_multiple(cluster, new_cluster, sim_fct)[0][-1,-1])
      time_mat += time() - start
    matrix = np.concatenate((matrix, np.array([new_column]).T), axis=1)
    matrix = np.concatenate((matrix, np.zeros((1, len(clusters)))), axis=0)

  print("Time for matrix computation: ", time_mat)
  print("Time for trace back: ", time_trace)
  
  return clusters[0], D[-1,-1]

In [55]:
# test
seq = ["CAT", "CHAT", "HER"]

res = progressive_alignment(seq, sim_blossum)

print(res)

Time for matrix computation:  0.0005555152893066406
Time for trace back:  2.4557113647460938e-05
(['-HER', 'C-AT', 'CHAT'], -10.0)


In [56]:
from Bio import SeqIO

def read_fasta(filename):
    """
    Read a fasta file and return the sequences as a list of strings
    """
    sequences = []
    for record in SeqIO.parse(filename, "fasta"):
        sequences.append(str(record.seq))
    return sequences

In [57]:
seq = read_fasta("data/paralogs.fasta")
test = progressive_alignment(seq, sim_blossum)
for s in test[0]:
  print(s)
print(f"Score of alignment: {test[1]}")

Time for matrix computation:  11.2442045211792
Time for trace back:  0.0018198490142822266
MG-LSDGEWQLVLNVWGKVEADIPGHGQEVLIRLFKGHPETLEKFDKFKHLKSEDEMKASEDLKKHGATVLTALGGILKKKGHHEAEIKPLAQSHATKHKIPVKYLEFISECIIQVLQSKHPGDFGADAQGAMNKALELFRKDMASNYKELGFQG
MVHLTPEEKSAVTALWGKV--NVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAH--K---YH-
MVHLTPEEKTAVNALWGKV--NVDAVGGEALGRLLVVYPWTQRFFESFGDLSSPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFSQLSELHCDKLHVDPENFRLLGNVLVCVLARNFGKEFTPQMQAAYQKVVAGVANALAH--K---YH-
MVHFTAEEKAAVTSLWSKM--NVEEAGGEALGRLLVVYPWTQRFFDSFGNLSSPSAILGNPKVKAHGKKVLTSFGDAIKNMDNLKPAFAKLSELHCDKLHVDPENFKLLGNVMVIILATHFGKEFTPEVQAAWQKLVSAVAIALAH--K---YH-
MGHFTEEDKATITSLWGKV--NVEDAGGETLGRLLVVYPWTQRFFDSFGNLSSASAIMGNPKVKAHGKKVLTSLGDAIKHLDDLKGTFAQLSELHCDKLHVDPENFKLLGNVLVTVLAIHFGKEFTPEVQASWQKMVTAVASALSS--R---YH-
MS-LTKTERTIIVSMWAKISTQADTIGTETLERLFLSHPQTKTYFPHF-DLH-P----GSAQLRAHGSKVVAAVGDAVKSIDDIGGALSKLSELHAYILRVDPVNFKLLSHCLLVTLAARFPADFTAEA

## Test protein with PDB: brouillon

In [59]:
from Bio.PDB import *

parser = PDBParser()
structure = parser.get_structure("1MBD", "data/1mbd.pdb")

for model in structure:
    print(model)
    for chain in model:
        print(chain)
        for residue in chain:
            # print(residue)
            # print(residue.get_resname(), residue.get_id()[1], chain.id)
            pass

#dssp = DSSP(model, )

<Model id=0>
<Chain id=A>
